<a href="https://colab.research.google.com/github/Androidex147/Clasificador-Perro-Gato/blob/main/Clasificador_perros_gatos_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Clasificador perro / Gato

Codigo que necesitaremos ejecutar para que el clasificador funcione


In [ ]:
import numpy as np

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
from google.colab import drive
#Para que pueda abrir el drive y ver las imagenes
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Si se quiere modificar habra que generar en teacheble machine un nuevo moedelo para que distinga en lo que queramos. Despuesde hacer el modelo en la pagina hay que exportarlo y cuandi¡o lo haga nos descargara dos archivos uno keras_model.h5 y labels.txt estos dos hay que guardarlos(preferiblmente dentro de una carpeta de drive).

In [ ]:
from tensorflow import keras
model=keras.models.load_model('/content/drive/MyDrive/IA Dogs vs Cats/keras_model.h5',compile=(False))
#Añadir la ruta de el archivo keras_model.h5
nombres_clases=open('//content/drive/MyDrive/IA Dogs vs Cats/labels.txt','r').readlines()
#añadir la ruta donde esta el archivo labels.txt
print(nombres_clases)
#Codigo para establecer la clasificacion perro/gato

['0 Gatos\n', '1 Perros\n']


##Codigo que identifica si es un perro o un gato

######Este código carga un modelo de Keras pre-entrenado para clasificar imágenes. Luego, toma una imagen específica de perro, la preprocesa (redimensiona y normaliza), y usa el modelo para predecir si la imagen es de un perro o un gato.

In [ ]:
from PIL import Image,ImageOps
from tensorflow import keras

model=keras.models.load_model('/content/drive/MyDrive/IA Dogs vs Cats/keras_model.h5',compile=(False))
#Añadir la ruta de el archivo keras_model.h5
nombres_clases=open('//content/drive/MyDrive/IA Dogs vs Cats/labels.txt','r').readlines()
#añadir la ruta del archivo labels.txt

imagen= Image.open("/content/drive/MyDrive/IA Dogs vs Cats/Gatos y perros prueba/dog.9783.jpg").convert("RGB")
#Cargar la ruta de la imagen que quieres que analize
size = (224, 224)
imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
imagen_array = np.asarray(imagen)
#Convertir la imagen al tamaño indicado y hacerla un array de Numpy
normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1
#Crear un array para un lote de 1 imagen. ndarray = N-Dimensional Array
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
lote_imagenes[0] = normalizada_imagen_array
resultados= model.predict(lote_imagenes)
indice = np.argmax(resultados[0])
etiqueta = nombres_clases[indice]
print("La imagen es de clase: ", etiqueta)
probabilidad = resultados[0][indice]
print("Con una probabilidad de: ", probabilidad)

1/1 [==============================] - 3s 3s/step
La imagen es de clase:  1 Perros

Con una probabilidad de:  1.0


##Codigo que identifica si es un perro o un gato (todas las imagenes)

Este código evalúa el rendimiento de tu modelo de clasificación. Recorre todas las imágenes en una carpeta específica, predice si cada imagen es un perro o un gato usando la función predecir_imagen, y compara la predicción con la etiqueta esperada (basada en el nombre del archivo). Finalmente, calcula la precisión del modelo y lista las imágenes que fueron clasificadas incorrectamente, junto con su probabilidad de predicción.

In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np

def predecir_imagen(ruta_imagen):
    imagen = Image.open(ruta_imagen).convert("RGB")
    size = (224, 224)
    imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
    imagen_array = np.asarray(imagen)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1
    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    lote_imagenes[0] = normalizada_imagen_array
    resultados = model.predict(lote_imagenes, verbose=0)
    indice = np.argmax(resultados[0])
    etiqueta_raw = nombres_clases[indice].strip()
    #Redimensionar las imagenes y hacerlas arrays de numpy

    if 'Gatos' in etiqueta_raw:
        etiqueta_predicha = "gato"
    elif 'Perros' in etiqueta_raw:
        etiqueta_predicha = "perro"
    else:
        etiqueta_predicha = "desconocido"

    probabilidad = resultados[0][indice]
    return etiqueta_predicha, probabilidad

lista_archivos = os.listdir("/content/drive/MyDrive/IA Dogs vs Cats/Gatos y perros prueba")
#cargar la ruta de la carpeta con todas la imagens que quieras analizar
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []

for nombre_archivo in lista_archivos:
    if 'cat' in nombre_archivo:
        etiqueta_esperada = "gato"
    elif 'dog' in nombre_archivo:
        etiqueta_esperada = "perro"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/IA Dogs vs Cats/Gatos y perros prueba", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)

precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos = media_probabilidad_aciertos / aciertos if aciertos > 0 else 0

informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Predicción: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)
#Recogemos las variables para con estas generar un informe del análisis de las imagenes


Informe de Evaluación del Modelo

Total de predicciones: 400
Aciertos: 390
Precisión: 97.50%
Probabilidad media de aciertos: 99.65%

Predicciones incorrectas:
Archivo: dog.9808.jpg, Predicción: gato, Probabilidad: 83.64%
Archivo: dog.9911.jpg, Predicción: gato, Probabilidad: 99.85%
Archivo: dog.9875.jpg, Predicción: gato, Probabilidad: 99.09%
Archivo: dog.9868.jpg, Predicción: gato, Probabilidad: 98.24%
Archivo: dog.9935.jpg, Predicción: gato, Probabilidad: 93.59%
Archivo: cat.9836.jpg, Predicción: perro, Probabilidad: 98.11%
Archivo: cat.9882.jpg, Predicción: perro, Probabilidad: 95.13%
Archivo: cat.9954.jpg, Predicción: perro, Probabilidad: 85.22%
Archivo: cat.9960.jpg, Predicción: perro, Probabilidad: 97.78%
Archivo: cat.9974.jpg, Predicción: perro, Probabilidad: 93.96%

